# Count Min and Top K heavy Hitters based leaderboard for Game of Thrones Characters

We obtain the elasticsearch index connected at localhost:9200 and stream 1000 lastest tweets that have used common Game of Thrones tags mentioned in the other iPython Notebook and reproduced here:
'#got', '#gameofthrones', '#winteriscoming', '#winterishere', '#winterfell', '#gotseason', '#asongoficeandfire'

The tweets that are streamed using these hahstags are indexed in Elasticsearch. We have implemented five DRPC queries, namely:
    1. Match Query
    2. Match Query with AND operation
    3. Aggregation Query
    4. Term Query
    5. Range Query

These will be explained as we run you through this application of Count Min Sketches and Bloom Filters to generate a leaderboard for Game of Thrones Characters. 
The implementation of Bloom Filters and Count Min Sketches is in two other python files named `BloomFilter.py` and `MinSketch.py`. Their implementation has been explained within these files.

In [32]:
from elasticsearch import Elasticsearch
from BloomFilter import *
from MinCount import *
from nltk.tokenize import word_tokenize
import string
import time

After we import all requires libraries and modules we need to first connect with elasticsearch.

In [33]:
es_conn = Elasticsearch('localhost:9200')

Let us now go through the DRPC query implemented here.
#### Match Query
    This query matches the exact same keyword that has been input into the function and delivers 1000 ES queries ordered in a descending manner of their creation.

In [34]:
def match_query(keyword):
    query={
        "query":{
            "match":{
                "text":keyword
            }
        },
        "sort": [
        {
          "timestamp": {
            "order": "desc"
          }
        }
      ]
    }
    response = es_conn.search(index= 'stock', body = query, size=1000)
    return response
    if response.get('hits', {}).get('total', 0) > 0:
        return response['hits']['hits'][0]['_source']['text']

### Application of Bloom Filter and Count Min Sketches in generating a leaderboard for Game of Thrones characters

Here we have 31 popular characters of Game of Thrones using whose names we have generated a Bloom Filter. Considering an error rate of 1%, we calculated the size of the Bloom Filter and the Count Min Sketch as such:  

    n = 31  
    p = 0.01  
    m = 300 (approximated from 297.13 ~ 298)  
    k = 7  
    Hence we have a min sketch & bloom filter of length 300 and it uses 7 hash functions.  

The manner of execution of this leaderboard involves first preprocessing the input twitter text to remove punctuation and converting it to lower case, after which it must be tokenised. The Bloom Filter is created and cross checked to determine if the current word is contained in the bloom filter. In case it is, its Count Min Sketch is updated to reflect this. The leaderboard is a dictionary that is updated with every new instance of a word referenced in the boom filter.

In [35]:
while True:    
    bloom = BloomFilter(31, 300, 7)
    bloom.createBloom()
    CMS = MinCount(300, 7)
    names = ["ned", "eddard", "robert", "jamie", "catelyn", "cersei", "daenerys", "jorah", "viserys", "jon", "sansa", "arya", "theon", "bran", "joffrey", "tyrion", "baelish", "davos", "samwell", "stannis", "melisandre", "jeor", "bronn", "varys", "tywin", "gendry", "brienne", "ramsay", "gilly", "daario", "missandei"]

    leaderboard = {i: 0 for i in names}

    query_res = match_query("got")['hits']['hits']
    
    final = []
    for value in query_res:
        text = str(value['_source']['text'].encode('utf-8')).lower().translate(None, string.punctuation)
        word_tokens = word_tokenize(text)

        for word in word_tokens:
            if bloom.checkBloom(word):
                CMS.insertCMS(word)
                c = CMS.count(word)
                leaderboard[word] = c
                filtered_words.append(word)

    sorted_leaderboard = sorted((value, key) for (key,value) in leaderboard.items())
    print sorted_leaderboard[-5:]
    time.sleep(5)

[(27, 'samwell'), (41, 'bran'), (60, 'got\xe2\x80\xa6'), (86, 'cersei'), (176, 'jon')]
[(27, 'samwell'), (41, 'bran'), (60, 'got\xe2\x80\xa6'), (86, 'cersei'), (176, 'jon')]
[(27, 'samwell'), (41, 'bran'), (60, 'got\xe2\x80\xa6'), (86, 'cersei'), (176, 'jon')]
[(27, 'samwell'), (41, 'bran'), (60, 'got\xe2\x80\xa6'), (86, 'cersei'), (176, 'jon')]
[(27, 'samwell'), (41, 'bran'), (60, 'got\xe2\x80\xa6'), (86, 'cersei'), (176, 'jon')]
[(27, 'samwell'), (41, 'bran'), (60, 'got\xe2\x80\xa6'), (86, 'cersei'), (176, 'jon')]
[(27, 'samwell'), (41, 'bran'), (60, 'got\xe2\x80\xa6'), (86, 'cersei'), (176, 'jon')]
[(27, 'samwell'), (41, 'bran'), (60, 'got\xe2\x80\xa6'), (86, 'cersei'), (176, 'jon')]
[(27, 'samwell'), (41, 'bran'), (60, 'got\xe2\x80\xa6'), (86, 'cersei'), (176, 'jon')]
[(27, 'samwell'), (41, 'bran'), (60, 'got\xe2\x80\xa6'), (86, 'cersei'), (176, 'jon')]
[(27, 'samwell'), (41, 'bran'), (60, 'got\xe2\x80\xa6'), (86, 'cersei'), (176, 'jon')]
[(27, 'samwell'), (41, 'bran'), (60, 'got\x

KeyboardInterrupt: 